In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
import os

In [ ]:
df = pd.read_csv(r'train.csv')

In [ ]:
le = LabelEncoder()
df['Class'] = le.fit_transform(df['Class'])

In [ ]:
test = pd.read_csv(r'test.csv')

In [ ]:
for i in range(df.shape[0]):
    df.iloc[i, 0] = "Train Images/"+df.iloc[i, 0]

In [ ]:
for i in range(len(test)):
    test.iloc[i, 0] = "Test Images/"+test.iloc[i, 0]

In [ ]:
import matplotlib.image as img

In [ ]:
class SplitDataset(Dataset):
    def __init__(self, data, transform = None):
        super().__init__()
        self.data = data.values
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        img_path, label = self.data[index]
        image = img.imread(img_path)
#         print("Before", image.shape)
        if self.transform is not None:
            image = self.transform(image)
#         print(image.shape)
        return image, label

In [ ]:
im_d = 224

In [ ]:
train_transform = transforms.Compose([transforms.ToPILImage(), transforms.Resize((im_d, im_d)),
                                    transforms.Grayscale(),
                                     transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5], std=[0.5])])
valid_transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize((im_d, im_d)),transforms.Grayscale(),
                                     transforms.ToTensor(), transforms.Normalize(mean=[0.5], std=[0.5])])
                                     #transforms.Normalize(mean=[0.485, 0.456, 0.406],
#        std=[0.229, 0.224, 0.225])])
test_transform = transforms.Compose([transforms.ToPILImage(),transforms.Resize((im_d, im_d)),transforms.Grayscale(),
                                     transforms.ToTensor(),
                                     transforms.Normalize(mean=[0.5], std=[0.5])])

In [ ]:
train, validate = train_test_split(df, test_size=0.2)

In [ ]:
train_data = SplitDataset(train, train_transform)
valid_data = SplitDataset(validate, valid_transform)
test_data = SplitDataset(test, test_transform)

In [ ]:
epochs = 2
batch_size = 32
n_classes = 4
learning_rate = 0.001

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)#, #num_workers=0)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)#, #num_workers=0)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)#, num_workers=0)

In [ ]:
import torch.nn as nn

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=10, kernel_size=3)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=3)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(1, 1024)
        self.fc2 = nn.Linear(1024, 4)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
#         print(x.shape)
        x = x.view(x.shape[0], -1)
        print(x.shape)
        x = x.unsqueeze(-1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
#         print(x)
        return x

In [ ]:
model = CNN()

In [ ]:
train_losses = []
test_losses = []

In [ ]:
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [ ]:
import time

In [ ]:
for i in range(epochs):
    start = time.time()
    train_loss = 0.0
    test_loss = 0.0
    model.train()
    for data, target in train_loader:
        data = data.to(device)
#         print(target)
        target = torch.Tensor(target).to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * data.size(0)
        
    model.eval()
    for data, target in valid_loader:
        data = data.to(device)
        target = torch.Tensor(target).to(device)
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss.item() * data.size(0)
        
    train_loss = train_loss/len(train_loader.sampler)
    test_loss = test_loss/len(valid_loader.sampler)
    end = time.time()
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f} \tTime: {:.6f}'.format(
        i, train_loss, test_loss, (end-start)))

In [ ]:
ok = test[:5]

In [ ]:
pred = test_transform(img.imread(test.iloc[0, 0]))

In [ ]:
out = model(pred)